In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4 
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)
max_len = 0


def prepare_training(path, n_runs):
    
    def get_max_len(sequence_list):
        m_len = 0
        for seq in sequence_list:
            if len(seq) > m_len:
                m_len = len(seq)
        max_len = m_len
        print(max_len)
        return m_len
    
    
    def construct_matrix(sequence_list):
        max_len = get_max_len(sequence_list)
        max_len = 640
        train_matrix = np.zeros(shape=(n_runs, max_len, n_features))
        for index,run in enumerate(sequence_list):
            train_matrix[index, :len(run)] = run
        print(train_matrix.shape)
        return train_matrix
        
        
    def stadard_sequences(sequences):
        for i, seq in enumerate(sequences):
            sequences[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
        return sequences       
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        stand_sequences = stadard_sequences(run_list_mix)
        padded_matrix = construct_matrix(stand_sequences)
        return padded_matrix
    
    
    
    return read_sequences()
    

train_matrix = prepare_training("Mix_sequences_var_length/run^", n_runs=200) 


629
(200, 640, 4)


In [19]:

from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose
from keras.losses import mse
from keras.models import Model


filters = 50
intermediate_dimension = 30 
latent_dim = 10

def Conv1DTranspose(input_tensor, filters, kernel_size, strides=2, padding='same'):
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            activation='relu',strides=(strides, 1), padding='valid')(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x



def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def repeat(x):
    steps_matrix = K.ones_like(x[0][:, :, :1])
    
    latent_matrix = K.expand_dims(x[1], axis=1)
    return K.batch_dot(steps_matrix, latent_matrix)


def create_vae():
    print(max_len)
    inputs = Input(shape=(624, n_features))
    x = inputs
    
    for i in range(2):
        x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                   padding='valid')(x)
    
    shape = K.int_shape(x)
    x = LSTM(intermediate_dimension)(x)
    embeddings = Dense(latent_dim)(x)
    
    z_mean = Dense(latent_dim, name='z_mean',)(embeddings)
    z_log_var = Dense(latent_dim, name='z_log_var')(embeddings)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    #x = Lambda(repeat)([before_flattening, z])
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    
    print(K.int_shape(x))
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x,
                            filters=filters,
                            kernel_size=2,
                            last=False,
                            padding='valid')
    
    
    output = Dense(n_features)(x)
    
    encoder.summary()
    decoder = Model(latent_inputs, output)
    decoder.summary()
    outputs = decoder(encoder.outputs[2])
    reconstruction_loss = K.categorical_crossentropy(K.flatten(inputs), K.flatten(outputs))
    #reconstruction_loss *= sequence_length*4
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), 
                             axis=-1)
    loss = K.mean(reconstruction_loss+kl_loss)
    vae = Model(inputs, outputs, name='vae')
    vae.add_loss(loss)
    #vae.summary()
    vae.compile(optimizer='rmsprop')
    return vae, encoder


def create_ae():
    inputs = Input(shape=(640, n_features))
    x = inputs
    for i in range(2):
        x = Conv1D(filters=filters, kernel_size=2, strides=2,
                   activation='relu',
                   padding='valid')(x)
    shape = K.int_shape(x)   
    x = LSTM(intermediate_dimension)(x)
    #x = Flatten()(x)
    encoded = Dense(latent_dim)(x)
    #x = Lambda(repeat)([before_flattening, encoded])
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x, filters=filters,
                            kernel_size=2, padding='valid')

       
    #decoded = LSTM(n_features, return_sequences=True)(x)
    
    output = Dense(n_features, activation='softmax')(x)
    
    encoder = Model(inputs, encoded)
    encoder.summary()
    decoder = Model(latent_inputs, output)
    
    output = decoder(encoder.output)
    sequence_autoencoder = Model(inputs, output)
    sequence_autoencoder.summary()
    sequence_autoencoder.compile(optimizer='rmsprop', 
                                 loss='categorical_crossentropy')
    return sequence_autoencoder, encoder


model, encoder = create_ae()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 640, 4)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 320, 50)           450       
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 160, 50)           5050      
_________________________________________________________________
lstm_6 (LSTM)                (None, 30)                9720      
_________________________________________________________________
dense_16 (Dense)             (None, 10)                310       
Total params: 15,530
Trainable params: 15,530
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_

In [ ]:
from keras.callbacks import ModelCheckpoint


def train():
    
    print(train_matrix.shape)
    model.fit(train_matrix,train_matrix, epochs=100, verbose=1)
    model.save_weights("Models/Weights/AE_CONV_LSTM_Diff_len_dist_MATRIX_LEN.hdf5")


train()


In [3]:

model.load_weights("Models/Weights/VAE_CONV_LSTM_Diff_len_dist_MATRIX_LEN.hdf5")


In [21]:

from sklearn.decomposition import PCA


encodings = encoder.predict(train_matrix)

#enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[2]
enc_mean, enc_var, z_enc = encodings, encodings, encodings

print(enc_mean.shape, enc_var.shape, z_enc.shape)


def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    
    
    
    return arg


masks = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]


from mpl_toolkits.mplot3d import Axes3D  



def plot_pca(title, i): 
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
    # for mask in unseen_mask:
    #     ax.scatter(unseen_encoding[0][:,0][mask],
    #                unseen_encoding[0][:,1][mask],
    #                unseen_encoding[0][:,2][mask])
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
    # for mask in unseen_mask:
    #     plt.scatter(unseen_encoding[0][:,0][mask],
    #            unseen_encoding[0][:,1][mask])
    #     
        #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    plot_pca('Sequences'+titles[i], 0)
    
    # principalComponents = enc
    # plot_pca('Sequences_Not_Pca'+titles[i], 0)
    


(200, 10) (200, 10) (200, 10)


(200, 3)
[0.54929274 0.37255278 0.04746724]


(200, 3)
[0.54929274 0.37255278 0.04746724]


(200, 3)
[0.54929274 0.37255278 0.04746724]


In [32]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)

print(tsne_obj.shape)

for mask in masks:
    plt.scatter(x=tsne_obj[:, 0][mask], 
                y=tsne_obj[:, 1][mask],
                alpha=0.5)
plt.show()


(200, 2)


In [12]:
unseen_labs = pd.read_csv("Data/Boat_nominal_data/Boat_unseen_labels_mix.csv")
unseen_labs = unseen_labs.drop(columns="Unnamed: 0") 
unseen_labs = np.array(unseen_labs)
unseen_mask = [return_mask(num, np.array(unseen_labs))[:, 0] for num in range(0, 9)]

unseen_sequences_matrix = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=200)

print(unseen_sequences_matrix.shape)
unseen_encoding = encoder.predict(unseen_sequences_matrix)
print(len(unseen_encoding), unseen_encoding[0].shape)


605
(200, 640, 4)
(200, 640, 4)


200 (10,)


In [23]:
#RECONSTRUCTION
def reconstruct_sequence(seq_index):
    run = train_matrix[seq_index]
    mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
    
    reconstr_run = model.predict(np.reshape(run, (1, run.shape[0], run.shape[1])))
    reconstr_run = np.reshape(reconstr_run, (run.shape[0], run.shape[1]))
    reconstr_run = reconstr_run[mask_seq]
    df = pd.DataFrame(reconstr_run, columns=["Sin", "Cosin", "Lat", "Lon"])
    plt.plot(df['Lon'], df['Lat'])
    plt.show()

for i in range(len(train_matrix)):
    reconstruct_sequence(i)
    if i > 5:
        break
    
reconstruction = model.predict(train_matrix)
reconstruction_unseen = model.predict(unseen_sequences_matrix)



In [24]:
#RECONSTRUCTION ERROR
def get_reconstructed_matrix(input_matrix, reconstrut):
    return_matrix = np.zeros(shape=input_matrix.shape)
    for i,run in enumerate(input_matrix):
        mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
        return_matrix[i][mask_seq] = reconstrut[i][mask_seq]    
    return reconstrut


train_error = abs(train_matrix-get_reconstructed_matrix(train_matrix,reconstruction))
unseen_runs_error = abs(unseen_sequences_matrix-get_reconstructed_matrix(train_matrix,reconstruction_unseen))

train_error_avg = np.mean(train_error, axis=2)
unseen_error_avg = np.mean(unseen_runs_error, axis=2)
train_error_avg = np.mean(train_error_avg, axis=1)
unseen_error_avg = np.mean(unseen_error_avg, axis=1)
print(train_error_avg.shape, unseen_error_avg.shape)

for mask in masks:
    plt.scatter(np.linspace(1,200,200)[mask],train_error_avg[mask])
plt.title('ERROR ON TRAIN')
plt.show()

for mask in unseen_mask:
    plt.scatter(np.linspace(1,200,200)[mask],unseen_error_avg[mask])
plt.title('ERROR ON Unseen')
plt.show()

(200,) (200,)


In [18]:
import ipyvolume as ipv
import numpy as np
x, y, z = unseen_encoding[:,0], unseen_encoding[:,1], unseen_encoding[:,2]

for mask in unseen_mask:
    ipv.scatter(x[mask], y[mask], z[mask], size=0.3, marker="sphere")
ipv.show()